#### Import libs and let's have a look on the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('/home/jupyter-v.hohlov-16/Product_folder/KC_case_data .csv', parse_dates = ['date'])
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [3]:
df.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [4]:
df.shape

(2747968, 8)

In [18]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

#### MAU in february:

In [5]:
df['month'] = df['date'].dt.month

In [6]:
df.query('month == 2')

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
919164,2020-02-01,app_start,NaN,android,27421,male,Saint-Petersburg,-,2
919165,2020-02-01,app_start,NaN,android,52191,female,Saint-Petersburg,-,2
919166,2020-02-01,app_start,NaN,android,57689,female,Saint-Petersburg,vk_ads,2
919167,2020-02-01,app_start,NaN,ios,61236,male,Saint-Petersburg,instagram_ads,2
919168,2020-02-01,app_start,NaN,android,118707,male,Saint-Petersburg,facebook_ads,2
...,...,...,...,...,...,...,...,...,...
2733177,2020-02-29,register,NaN,ios,567982,female,Saint-Petersburg,facebook_ads,2
2733178,2020-02-29,register,NaN,ios,19689619,female,Moscow,referal,2
2733179,2020-02-29,register,NaN,ios,345024,male,Moscow,yandex-direct,2
2733180,2020-02-29,register,NaN,android,33492617,male,Moscow,facebook_ads,2


In [30]:
df.query('month == 2').groupby('device_id').agg({'device_id': 'nunique'})

,device_id
device_id,
4014,1
4016,1
4046,1
4076,1
4120,1
...,...
35090266,1
35106965,1
35108206,1


#### Installs in january:

In [7]:
df.query('month == 1 & event == "app_install"')

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,1
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,1
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,1
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,1
...,...,...,...,...,...,...,...,...,...
2595353,2020-01-31,app_install,NaN,android,3606659,male,Moscow,google_ads,1
2595354,2020-01-31,app_install,NaN,ios,11501360,female,Saint-Petersburg,vk_ads,1
2595355,2020-01-31,app_install,NaN,android,10118705,male,Saint-Petersburg,vk_ads,1
2595356,2020-01-31,app_install,NaN,ios,19855521,female,Moscow,-,1


#### Divide users into installation day cohorts and calculate CR from install to 7-day purchase. Which cohort has the biggest CR? Believe that converted user is the user with 7-day period between install date and purchase date.

In [94]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


In [9]:
merged_df = df.query('event == "app_install"').merge(df.query('event == "purchase"'), on = 'device_id')
merged_df.head()

,date_x,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,month_x,date_y,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y,month_y
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-01-10,purchase,311.0,ios,male,Moscow,-,1
1,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-01-19,purchase,1921.5,ios,male,Moscow,-,1
2,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-02-05,purchase,322.5,ios,male,Moscow,-,2
3,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-02-11,purchase,513.5,ios,male,Moscow,-,2
4,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-03-15,purchase,131.5,ios,male,Moscow,-,3


In [10]:
merged_df['Difference'] = (merged_df['date_y'] - merged_df['date_x']).dt.days
merged_df

,date_x,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,month_x,date_y,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y,month_y,Difference
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-01-10,purchase,311.0,ios,male,Moscow,-,1,9
1,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-01-19,purchase,1921.5,ios,male,Moscow,-,1,18
2,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-02-05,purchase,322.5,ios,male,Moscow,-,2,35
3,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-02-11,purchase,513.5,ios,male,Moscow,-,2,41
4,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,1,2020-03-15,purchase,131.5,ios,male,Moscow,-,3,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115220,2020-03-31,app_install,NaN,ios,228794,female,Saint-Petersburg,facebook_ads,3,2020-03-31,purchase,473.5,ios,female,Saint-Petersburg,facebook_ads,3,0
115221,2020-03-31,app_install,NaN,android,20224705,female,Moscow,instagram_ads,3,2020-03-31,purchase,88.0,android,female,Moscow,instagram_ads,3,0
115222,2020-03-31,app_install,NaN,android,1434534,male,Moscow,referal,3,2020-03-31,purchase,525.5,android,male,Moscow,referal,3,0
115223,2020-03-31,app_install,NaN,android,271586,male,Moscow,-,3,2020-03-31,purchase,6502.0,android,male,Moscow,-,3,0


In [12]:
final = merged_df.query('Difference <= 7').groupby('date_x').agg({'device_id': 'nunique'}) / df.query('event == "app_install"').groupby('date').agg({'device_id': 'nunique'})
final.sort_values(by = 'device_id', ascending =  False)

,device_id
date_x,
2020-01-01,0.393406
2020-01-09,0.391854
2020-01-15,0.382831
2020-01-14,0.381403
2020-01-02,0.377226
...,...
2020-03-18,0.200683
2020-03-29,0.199642
2020-03-22,0.199048


#### Which paid marketing channel has the biggest amount of new users?

In [17]:
df.groupby('utm_source').agg({'device_id':'nunique'}).sort_values(by = 'device_id', ascending = False)

,device_id
utm_source,
-,52273
yandex-direct,40712
google_ads,38096
vk_ads,34086
instagram_ads,31048
facebook_ads,25959
referal,15926


#### Which stage of sales funnel has the most of signed users who left the product? Show stage-to-stage CR

##### At first add the column with registration data for dividing signed and non-signed users:

In [9]:
df['registration_date'] = df.query('event == "register"')['date']

In [11]:
df['date_diff'] = (df['date'] - df['registration_date']).dt.days

In [16]:
registered_users = df.query('date_diff == 0')

In [17]:
registered_users

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,registration_date,date_diff
2669658,2020-01-01,register,NaN,ios,294193,female,Saint-Petersburg,instagram_ads,2020-01-01,0.0
2669659,2020-01-01,register,NaN,ios,22917617,male,Moscow,google_ads,2020-01-01,0.0
2669660,2020-01-01,register,NaN,android,15248490,male,Moscow,-,2020-01-01,0.0
2669661,2020-01-01,register,NaN,android,252062,female,Moscow,vk_ads,2020-01-01,0.0
2669662,2020-01-01,register,NaN,android,2251583,female,Moscow,instagram_ads,2020-01-01,0.0
...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-31,0.0
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,0.0
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,0.0
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-31,0.0


##### At second merge datasets for adding all events per signed user:

In [20]:
reg_users_actions = df.merge(registered_users, on = 'device_id')

In [21]:
reg_users_actions

,date_x,event_x,purchase_sum_x,os_name_x,device_id,gender_x,city_x,utm_source_x,registration_date_x,date_diff_x,date_y,event_y,purchase_sum_y,os_name_y,gender_y,city_y,utm_source_y,registration_date_y,date_diff_y
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,NaT,NaN,2020-01-07,register,NaN,android,female,Moscow,-,2020-01-07,0.0
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,NaT,NaN,2020-01-07,register,NaN,android,female,Moscow,-,2020-01-07,0.0
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,NaT,NaN,2020-01-07,register,NaN,android,female,Moscow,-,2020-01-07,0.0
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,NaT,NaN,2020-01-07,register,NaN,android,female,Moscow,-,2020-01-07,0.0
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,NaT,NaN,2020-01-07,register,NaN,android,female,Moscow,-,2020-01-07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205961,2020-03-31,purchase,654.0,android,908531,male,Saint-Petersburg,yandex-direct,NaT,NaN,2020-03-31,register,NaN,android,male,Saint-Petersburg,yandex-direct,2020-03-31,0.0
2205962,2020-03-31,search,NaN,android,908531,male,Saint-Petersburg,yandex-direct,NaT,NaN,2020-03-31,register,NaN,android,male,Saint-Petersburg,yandex-direct,2020-03-31,0.0
2205963,2020-03-31,tap_basket,NaN,android,908531,male,Saint-Petersburg,yandex-direct,NaT,NaN,2020-03-31,register,NaN,android,male,Saint-Petersburg,yandex-direct,2020-03-31,0.0
2205964,2020-03-31,app_install,NaN,android,908531,male,Saint-Petersburg,yandex-direct,NaT,NaN,2020-03-31,register,NaN,android,male,Saint-Petersburg,yandex-direct,2020-03-31,0.0


##### At third calculate events per signed user and stage-to-stage CR:

In [30]:
reg_users_actions = reg_users_actions.groupby('event_x').agg({'device_id': 'count'})

In [38]:
sort_reg_users_actions = reg_users_actions.query('event_x == "search" | event_x == "choose_item" | event_x == "tap_basket" | event_x == "register"').sort_values(by = 'device_id', ascending = False)

In [39]:
sort_reg_users_actions['CR'] = sort_reg_users_actions['device_id'] / sort_reg_users_actions['device_id'].shift(1)

In [40]:
sort_reg_users_actions

,device_id,CR
event_x,,
search,565271,NaN
choose_item,440467,0.779214
tap_basket,322287,0.731694
register,78310,0.242982


#### Which channels have the lowest first-purchase CR?

In [109]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,registration_date,date_diff
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,NaT,NaN
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,NaT,NaN
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,NaN
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,NaT,NaN
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,NaT,NaN


In [154]:
first_purchase_devices = df.query('event == "purchase"').groupby(['utm_source', 'device_id'])\
                                .agg({'device_id': 'count'})\
                                .query('device_id == 1')\
                                .rename(columns={"device_id": "purchase_num"})\
                                
first_purchase_devices

purchase_num
utm_source    device_id              
-             4046                  1
              4234                  1
              4244                  1
              4572                  1
              4789                  1
...                               ...
yandex-direct 35245401              1
              35258588              1
              35266881              1
              35277913              1
              35308278              1

[58435 rows x 1 columns]

In [155]:
first_purchase_devices.reset_index(inplace = True)

In [164]:
first_purchase_devices = first_purchase_devices.groupby('utm_source').agg({'device_id': 'count'})

In [158]:
first_app_start_devices = df.query('event == "app_start"').groupby(['utm_source', 'device_id'])\
                                .agg({'device_id': 'count'})\
                                .query('device_id == 1')\
                                .rename(columns={"device_id": "app_start_num"})\
                                
first_app_start_devices

app_start_num
utm_source    device_id               
-             4018                   1
              4046                   1
              4067                   1
              4088                   1
              4212                   1
...                                ...
yandex-direct 35308278               1
              35358658               1
              35360863               1
              35367137               1
              35388218               1

[186557 rows x 1 columns]

In [159]:
first_app_start_devices.reset_index(inplace = True)

In [163]:
first_app_start_devices = first_app_start_devices.groupby('utm_source').agg({'device_id': 'count'})

In [171]:
df_CR_calc = pd.merge(first_purchase_devices, first_app_start_devices, left_index=True, right_index=True)

In [173]:
df_CR_calc['first_purchase_CR'] = df_CR_calc['device_id_x'] / df_CR_calc['device_id_y']
df_CR_calc.sort_values(by = 'first_purchase_CR')

,device_id_x,device_id_y,first_purchase_CR
utm_source,,,
yandex-direct,9603,33812,0.284012
google_ads,8960,31094,0.288158
-,12224,39826,0.306935
instagram_ads,7824,24159,0.323854
vk_ads,9040,26892,0.336159
facebook_ads,6421,19040,0.337237
referal,4363,11734,0.371825


#### Which channel has the biggest ROMI (Return on Marketing Investment)?

In [73]:
general_revenue = df.groupby('utm_source').agg({'purchase_sum': 'sum'})
general_revenue

,purchase_sum
utm_source,
-,21449749.5
facebook_ads,12249901.0
google_ads,12868276.0
instagram_ads,14546969.0
referal,8837044.5
vk_ads,16389652.5
yandex-direct,13915368.0


#### Add cost info for paid marketing channels into dataset:

In [74]:
general_revenue['marketing_cost'] = [0, 8590498, 10534878, 8561626, 0, 9553531, 10491707]

In [76]:
general_revenue['ROMI'] = (general_revenue['purchase_sum'] - general_revenue['marketing_cost']) / general_revenue['marketing_cost']

In [78]:
general_revenue.sort_values(by = 'ROMI', ascending = False)

,purchase_sum,marketing_cost,ROMI
utm_source,,,
-,21449749.5,0,inf
referal,8837044.5,0,inf
vk_ads,16389652.5,9553531,0.715560
instagram_ads,14546969.0,8561626,0.699090
facebook_ads,12249901.0,8590498,0.425983
yandex-direct,13915368.0,10491707,0.326321
google_ads,12868276.0,10534878,0.221493


#### Which paid marketing channel has the biggest fisrt median purchase? 

In [89]:
the_only_purchase_df = df.query('event == "purchase"').groupby(['utm_source', 'device_id']).agg({'device_id': 'count'}).query('device_id == 1').rename(columns={"device_id": "pur_num"})

In [90]:
the_only_purchase_df.reset_index(inplace = True)

In [99]:
the_only_purchase_df.shape

(58435, 3)

In [105]:
pd.merge(the_only_purchase_df, df.query('event == "purchase"')).groupby('utm_source').agg({'purchase_sum': 'median'}).sort_values(by = 'purchase_sum')

,purchase_sum
utm_source,
facebook_ads,388.00
google_ads,389.50
referal,393.00
vk_ads,393.00
yandex-direct,393.00
instagram_ads,394.75
-,400.00
